### 1. Searching for name entities and keywords in news

In [1]:
import pandas as pd
from mitie import *
from collections import Counter, OrderedDict
import re
from tokenization import *

### Running NER

In [3]:
path_to_uk_model = '../NER_models/uk_model.dat'
path_to_ru_model = '../NER_models/ru_model.dat'
news_filepath = '../data/may.csv'

In [4]:
ner_uk = named_entity_extractor(path_to_uk_model)
ner_ru = named_entity_extractor(path_to_ru_model)

In [5]:
news = pd.read_csv(news_filepath, nrows=1000)
news.columns

Index(['id', 'title', 'text', 'subtitle', 'link', 'domain', 'datetime',
       'views', 'created_at', 'category', 'language', 'domain_alias',
       'mycategory'],
      dtype='object')

In [6]:
names_uk_path = '../dicts/names_ukr.txt' 
names_ru_path = '../dicts/names_ru.txt' 
with open(names_uk_path) as f:
    names_uk = [name.strip() for name in f.readlines()]
    
with open(names_ru_path) as f:
    names_ru = [name.strip() for name in f.readlines()]
names = list(set(names_uk+names_ru))
names_string = '|'.join(names)

In [7]:
news['all_text'] = news.title.str.cat(news.text, sep='\n', na_rep = '')
news['all_text'] = news.all_text.str.strip()

### Finding all entities

In [8]:
# get list of all entities in every sentence of tokenized text
def find_entities(tokens, ner_model):
    entities_in_sentences = []
    for sentence in tokens:
        entities = ner_model.extract_entities(sentence)
        entities_in_sentences.append(entities)
    return entities_in_sentences

In [9]:
#run find_entities for all df 
def process_news(news, start=0, finish=1000, step=100):
    for k in range(start, finish, step):
        try:
            del news_part
        except:
            pass    
        news_part = news.iloc[k:k + step].copy()
        news_part['entities'] = news_part.apply(lambda row: \
                                                find_entities(row.tokenized, ner_uk) if row.language=='uk' \
                                                else find_entities(row.tokenized, ner_ru), axis=1)
        news.entities.update(news_part.entities)

In [10]:
%%time
news['tokenized'] = news.apply(lambda row: tokenize(row.all_text, row.language), axis=1)
news['entities'] = None

CPU times: user 786 ms, sys: 23.6 ms, total: 809 ms
Wall time: 1.04 s


In [11]:
%%time
process_news(news, start=0, finish=len(news), step=1000)

CPU times: user 5.04 s, sys: 196 ms, total: 5.24 s
Wall time: 5.49 s


### Functions for joining list into strings and splitting strings into lists

In [14]:
def get_only_kw_and_names(kw_and_ent):
    if pd.notna(kw_and_ent) and (kw_and_ent!=''):
        n = kw_and_ent.count('PERS')
        res = []
        for sent in kw_and_ent.split('<+>'):
                sent_num, kw, entities = sent.split('<#>', maxsplit=2)
                names_scores = re.findall(r'PERS<§§>(.*?)<§§>([\d\.]+)', entities)
                for ns in names_scores:
                    res.append((sent_num, kw, ns[0], ns[1]))
        if n!=len(res):
            print(kw_and_ent)
        return res
    return None

In [15]:
def names_and_kw_to_string(names_and_kw):
    if isinstance(names_and_kw, list):
        res = []
        for name in names_and_kw:
            res.append('<+>'.join(name))
        return '@+@'.join(res)
    return None

def names_and_kw_to_list(names_and_kw_str):
    if pd.notnull(names_and_kw_str):
        res = []
        for ent in names_and_kw_str.split('@+@'):
            res.append(ent.split('<+>'))
        return res
    return None

In [16]:
# save entitites list to string to write to csv file
def entities_to_string(entities, tokens):
    res = []
    for s in range(len(entities)):
        sent_res= []
        for ent in entities[s]:
            entity_text = " ".join(tokens[s][i] for i in ent[0])
            ent_range=str(ent[0]).strip('range')
            sent_res.append('<§§>'.join([ent_range, ent[1], entity_text, str(round(ent[2], 2))]))
        res.append('<;;>'.join(sent_res))
    return '<@@>'.join(res)


#parse entitites strings into lists 
def ent_string_to_list(entity):
    res = []
    for sent in entity.split('++'):
        sent_entities=[]
        for ent in sent.split(';;'):
            ent_parts = ent.split('::')
            sent_entities.append(ent_parts)
        res.append(sent_entities)
    return res

In [17]:
# get only sentences that have PERS entity and a keyword
# save sentence number, keywords and entities to string
def keywords_and_names(name_and_kw, entities):
    res = []
    sentences = name_and_kw.split('#')
    for i in range(len(sentences)):
        if sentences[i] not in ['NPR','NKW']:
            res.append('<#>'.join([str(i), sentences[i], entities[i]]))
    return '<+>'.join(res)


# find keywords in sentences with PERS entity
# save result to string
def find_keywords(sentences, entities):
    res=[]
    if isinstance(entities, list):
        for s in range(len(sentences)):
            if 'PERS' in entities[s]:
                kw = re.search(keywords, sentences[s], flags=re.I)
                if kw:
                    res.append(kw.group())
                else:
                    res.append('NKW')
            else:
                res.append('NPR')
    else:
        res.append('NPR')
    return '#'.join(res)

In [18]:
news['all_ent_str'] = news.apply(lambda row: entities_to_string(row.entities, row.tokenized), axis=1)

In [19]:
news['sentenized'] = news.apply(lambda row: text_to_sent(row.all_text, row.language), axis=1)

news['ent_sent'] = news.all_ent_str.str.split('<@@>')

news['name_and_kw'] = news.apply(lambda row: find_keywords(row.sentenized, row.ent_sent), axis=1)

news['kw_and_ent'] = news.apply(lambda row: keywords_and_names(row.name_and_kw, row.ent_sent), axis=1)

news['names_and_kw'] = news.kw_and_ent.apply(get_only_kw_and_names)

news['names_and_kw_str'] = news.names_and_kw.apply(names_and_kw_to_string)

In [22]:
news.columns

Index(['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'domain_alias', 'all_text',
       'tokenized', 'entities', 'all_ent_str', 'sentenized', 'ent_sent',
       'name_and_kw', 'kw_and_ent', 'names_and_kw', 'names_and_kw_str',
       'mycategory'],
      dtype='object')

In [23]:
news['kw_and_ent'] = news.kw_and_ent.apply(lambda x: None if x=='' else x)

In [24]:
news['names_str'] = news.names_and_kw.apply(lambda ents:
                                                '§'.join([e[2] for e in ents]) if isinstance(ents, list) else None)

### Keywords

In [18]:
keywords_dict = ['(е|э)ксперт',
'анал(і|и)тик',
'(про)?анал(і|и)з',
'розпові(в|ла|дає|дав|дала)',
'(расс|роз|выс?|вис?|до|в|у|с)?каз(ав|ала?|ати|увати|ував|увала|ывала?|ывать|ує|ывает)',
'відсто(ює|яв|яла|ював|ювала)',
'(за|объ)яв(ила?|ив|ляє|ляет|ляв|ляла?)',
'пишет?',
'(на)?писа(в|ла?)',
'повідом(ила|ив|ляє|ляв|ляла)',
'посила(нням?|ється|вся|лася)',
'(про)?комм?ент(ував|ує|увала|ирует|ировала?|ар)',
'констат(ує|ував|увала|ирует|ировала?)',
'зауваж(ив|ила|ує|ував|увала)',
'зверну(в|ла)(с(я|ь))',
'звер(тав|таєть|тала)(с(я|ь))',
'звер(нув|нула|тає|тала|тав)\sувагу',
'акцент(ує|ував|увала|ирует|ировала?)',
'вислов(ив|ила|ити|лювала|лював)(с(я|ь))?',
'в(и|ы)с(унув|ував|унула?|увала)',
'вважа(є|в|ла)',
'говор(ить?|ив|ила?)',
'застеріг(ає|ав|ала)?',
'застерегла'
'запевн(яє|яла|яв|ив|ила)',
'(у|в)певнен(ий|а)',
'(с|під)тверд(жує|жував|жувала|ив|ила)',
'(у|под)твержд(ает|ала?)',
'(у|под)твердила?',
'(за|від)знач(ив|ила|ає|ав|ала)',
'каже',
'(по)?рад(ить|ив|ила)',
'(до|при)трим(ав|ала|уєть|ував|увала)с(я|ь)\sдумк',
'дум(ає|ав|ает|ала?)',
'за\s(словами|оцінк)',
'згідно\sз',
'согласно'
'відповідно\sдо',
'до(да|бав)(в|ила?|ла|є|ляет)',
'(на|о)голо(шує|сив|сила|шував|шувала|шувати)',
'наз(ы|и)?ва(ет|є|в|ла?)',
'з?роби(в|ла|ть)\s(виснов|заяв)',
'перекон(ує|ав|ував|ала|увала|аний|ана)',
'на\sдумку',
'(про)?цит(ує|ат|ував|увала?)',
'озвуч(ив|ла?)',
'підкресл(ює|ив|ила|ював|ювала)',
'(від|під)мі(чає|тив|тила|чав|чала)',
'спостер(ігав|ігала|еження|ігає)',
'визна(є|в|ла|вав|вала|ти|вати)',
'с?прогноз(ує|ував|увала|ировала?|ирует)',
'п(і|о)дозр(ює|евает|ював|ювала|евала?)',
'(при|до)пу(скає|скав|стив|скала|стила|щення|стити)',
'(за)?пропон(ував|увала)',
'дов(ів|ела|одить|одила|одив)',
'дон(іс|есла|осить?|осив|осила|ести|ес)',
'(по)?рекоменд(ує|ував|ац|увала|ует|овала?)',
'аргумент(ує|ував|увала|ирует|ировала?)?',
'повтор(ив|ила?|ює|ювала|ював)',
'дізна(ли|в|ла|вав|вали|вала)(с(я|ь))',
'(за|роз|с)пит(али|итав|ала)',
'уточн(ив|ила?|ював|ення|ювала)',
'призы?в(ала?|ает)',
'заклика(є|в|ла)',
'(от|под|за)ме(тила?|чает|чала?)',
'по\sмнению',
'по\sсловам',
'п(о|і)дтвер(ждает|джує|див|джував|джувала|дила?|ждала?)',
'сообщ(ать|ила?|ение)',
'р(о|а)збир(ала?|ав|али)(с(я|ь))?',
'р(о|а)з(і|о)бр(ала?|ав|али)(с(я|ь))?',
'(рас)?спросил',
'узна(вал|л)',
'по\s(оценк|мнен)',
'счита(ет|ю|ла?)',
'рассчитыва(ет|ю|ла?)',
'рассчитала?',
'розрахову(є|вав|вала)',
'(роз|по)раху(вав|вала)',                 
'(рас|о)цен(ивает|ивала?|ила?)',
'(роз|о)цін(ює|ював|ювала|ив|ила)',
'(по)?совет(ует|овала?)',
'подчерк(нула?|ивает)',
'ссыл(кой|ается|ался|алась)',
'обра(ти|ща)(л.?с)',
'обра(ти|ща)(ла?|ет)\sвниман',
'предостере(г|гла|гать|гала?|жен)',
'(у|за)вер(ена?|ять|яла?|ила?)',
'(при|до)держ(ивать|ала?|ивала?)с(я|ь)',
'(по|объ)ясн(ила?|яла?|ить|ять|ение)',
'с?дел(ать|ала?|ает)\s(вывод|заяв)',
'при(шел|шла|ходит)\sк\sвывод',
'заключ(ила?|ает)',
'убеж?д(ена?|ает|ала?|ила?)',
'наблюд(ает|ала?)',
'призна(ет|ла?|вала?)',
'пред(по)?л(аг|ож)(ает|ала?|ила?|ени)',
'(от|на)стаив(ает|ала?)',
'повтор(яет|ила?|яла?)',
'в(ы|и)ра(ж|з)(ила?|ает|ає|ив|ав|ала?)',
'рассу(ждает|ждала?|дила?)',
'(роз)?мірк(овує|овував|овувала)',
'підсумува(в|ла)',
'подытожила?',
'п?о?зна(к.м|йо)(ила?|мив|мила|млював|млювала|ливала?)'
]
keywords = '|'.join([r'\b' + kw for kw in keywords_dict])

### Saving

In [29]:
news.columns

Index(['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'domain_alias', 'all_text',
       'tokenized', 'entities', 'all_ent_str', 'sentenized', 'ent_sent',
       'name_and_kw', 'kw_and_ent', 'names_and_kw', 'names_and_kw_str',
       'mycategory', 'names_str', 'ent_strings'],
      dtype='object')

In [27]:
news[['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'domain_alias', 
       'mycategory']].to_csv(news_filepath)


In [28]:
news['ent_strings'] = news.apply(lambda row: entities_to_string(row.entities, row.tokenized), axis=1)

In [35]:
entities_path = '../data/entities_may.csv'
news[['link', 'language', 'all_ent_str', 
       'name_and_kw', 'kw_and_ent', 'names_and_kw_str',
       'names_str']].to_csv(entities_path)

In [160]:
news = news.drop(columns=['all_text'])

In [26]:
news.columns

Index(['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'id', 'domain_alias', 'all_text',
       'tokenized', 'entities', 'all_ent_str', 'sentenized', 'ent_sent',
       'name_and_kw', 'kw_and_ent', 'names_and_kw', 'names_and_kw_str',
       'names_str'],
      dtype='object')